In [1]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text

pdf_text = extract_text_from_pdf("./flipkart_jacket_invoice.pdf")

In [2]:
print(pdf_text)

E. & O.E.
page 1 of 1
*Keep this invoice and 
manufacturer box for 
warranty purposes.
Ship To
Prafull Narayan Sonawane 
Talathi Colony near govind nagar 
plot no. 39, Talathi Colony near 
govind nagar plot no. 39.
Pachora 424201 Maharashtra 
Phone: xxxxxxxxxx 
Bill To
Prafull Narayan Sonawane 
Talathi Colony near govind nagar 
plot no. 39, Talathi Colony near 
govind nagar plot no. 39.
Pachora 424201 Maharashtra 
Phone: xxxxxxxxxx 
Order ID: 
OD329836817381305100
 02-12-2023
Order Date:
 02-12-2023
Invoice Date:
 AAICA4872D
PAN:
 U52100HR2010PTC068415
CIN:
Invoice Number # FAETLP2402126539
Tax Invoice
Sold By: 
 
Tech-Connect Retail Private Limited ,
Ship-from Address: Marasandra and Madnahatti -Venkatapura Villages, Kasaba Hobli, Malur Taluk ,Dist -Kolar, Malur, Bangalore , 
Karnataka, India - 563130, IN-KA ,
GSTIN - 29AAICA4872D1ZK
Total items: 1
Product
Title
Qty
Gross
Amount ₹
Discounts
/Coupons ₹
Taxable
Value ₹
IGST ₹
Total ₹
Men Jackets
FSN: 
JCKFWWZJGEGZC7PY
HSN/SAC: 61043300


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate


from dotenv import load_dotenv
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [4]:
from langchain.prompts import ChatPromptTemplate

In [5]:
system = '''You are an Assistance bot who is good at understanding text, specifically invoices. The user will provide you with invoice text, and your task is to convert it to JSON format.
extract info like menu, sub_total, total

For eg.

{{
   'menu': [{{'nm': '<>',
   'cnt': {{'nm': '<>',
    'unitprice': '<>'}},
   'unitprice': '<>',
   'price': '<>'}},
  {{'nm': '<>',
   'cnt': {{'unitprice': '<>'}},
   'price': {{'cnt': '<>', 'unitprice': '<>'}},
  {{'nm': '<>',
   'unitprice': '<>',
   'cnt': '<>',
   'price': '<>'}},
  {{'cnt': '<>', 'price': '<>'}}],

 'sub_total': {{'subtotal_price': {{'cnt': '<>'}},

 'total': {{'total_price': '<>',
  'cashprice': '<>',
  'changeprice': '<>',
  'creditcardprice': '<>',
  'menuqty_cnt': '<>'}}
}}

'''


In [6]:
prompt_template = ChatPromptTemplate([
    ("system", system),
    ("human", "invoice data : {invoice_data}")
])


import json
def output(data):
    return json.loads(
        data
        .content
        .replace("```", "")
        .replace("json", "")
    )

In [7]:
chaining = (
    prompt_template |
    model |
    output
)

In [8]:
output_data = chaining.invoke(input={"invoice_data" : pdf_text})
output_data

{'menu': [{'nm': 'HRX by Hrithik Roshan Full Sleeve Solid Men Jacket',
   'cnt': 1,
   'unitprice': 1079.0,
   'price': 1079.0,
   'Discounts/Coupons': 0.0,
   'Taxable Value': 963.39,
   'IGST': 115.61},
  {'nm': 'Shipping And Handling Charges',
   'cnt': 1,
   'unitprice': 50.0,
   'price': 10.0,
   'Discounts/Coupons': -40.0,
   'Taxable Value': 8.93,
   'IGST': 1.07}],
 'sub_total': {'subtotal_price': {'cnt': 1129.0}},
 'total': {'total_price': 1089.0,
  'cashprice': None,
  'changeprice': None,
  'creditcardprice': None,
  'menuqty_cnt': 2}}

In [9]:
import pandas as pd

pd.json_normalize(output_data['menu'])

,nm,cnt,unitprice,price,Discounts/Coupons,Taxable Value,IGST
0,HRX by Hrithik Roshan Full Sleeve Solid Men Ja...,1,1079.0,1079.0,0.0,963.39,115.61
1,Shipping And Handling Charges,1,50.0,10.0,-40.0,8.93,1.07


In [10]:
pd.json_normalize(output_data['sub_total'])

,subtotal_price.cnt
0,1129.0


In [11]:
pd.json_normalize(output_data['total'])

,total_price,cashprice,changeprice,creditcardprice,menuqty_cnt
0,1089.0,None,None,None,2


In [22]:
# json.loads(output_data.content.replace("```", "").replace("json", ""))

In [57]:
# import json
# json.loads(output.content.replace("```", "").replace("json", ""))

In [94]:
# json_data = {
#   "retailer_info": {
#     "name": "Tech-Connect Retail Private Limited",
#     "address": "Marasandra and Madnahatti -Venkatapura Villages, Kasaba Hobli, Malur Taluk ,Dist -Kolar, Malur, Bangalore , Karnataka, India - 563130, IN-KA",
#     "gst": "29AAICA4872D1ZK",
#     "pan": "AAICA4872D",
#     "cin": "U52100HR2010PTC068415",
#     "regd_office": "Rectangle No. 06, Rectangle No. 07, Rectangle No. 08 and Rectangle No. 13, Village- Khalikpur, Tehsil- Badli, District- Jhajjar,FC- Farrukhnagar, Jhajjar, Haryana - 124103"
#   },
#   "customer_info": {
#     "name": "Prafull Narayan Sonawane",
#     "address": "Talathi Colony near govind nagar plot no. 39, Talathi Colony near govind nagar plot no. 39, Pachora 424201 Maharashtra",
#     "phone": "xxxxxxxxxx"
#   },
#   "invoice_info": {
#     "order_id": "OD329836817381305100",
#     "order_date": "02-12-2023",
#     "invoice_date": "02-12-2023",
#     "invoice_number": "FAETLP2402126539"
#   },
#   "products_details": [
#     {
#       "title": "HRX by Hrithik Roshan Full Sleeve Solid Men Jacket",
#       "fsn": "JCKFWWZJGEGZC7PY",
#       "hsn_sac": "61043300",
#       "quantity": 1,
#       "gross_amount": 1079.00,
#       "discount": 0.00,
#       "taxable_value": 963.39,
#       "igst": 115.61,
#       "total": 1079.00
#     },
#     {
#       "title": "Shipping And Handling Charges",
#       "quantity": 1,
#       "gross_amount": 50.00,
#       "discount": -40.00,
#       "taxable_value": 8.93,
#       "igst": 1.07,
#       "total": 10.00
#     }
#   ],
#   "totals": {
#     "total_items": 1,
#     "sub_total": 1129.00,
#     "total_discount": -40.00,
#     "total_taxable_value": 972.32,
#     "total_igst": 116.68,
#     "grand_total": 1089.00
#   }
# }

In [24]:
import pandas as pd
pd.set_option("display.max_columns", None)

def json2df(json_data) -> pd.DataFrame:
    """
    Convert JSON invoice data into a single Pandas DataFrame.
    """
    # Extract common fields
    retailer_info = json_data["retailer_info"]
    customer_info = json_data["customer_info"]
    invoice_info = json_data["invoice_info"]
    totals = json_data["totals"]
    
    # Extract product details and merge with common fields
    records = []
    for product in json_data["products_details"]:
        record = {
            "Retailer Name": retailer_info["name"],
            "Retailer Address": retailer_info["address"],
            
            "Customer Name": customer_info["name"],
            "Customer Address": customer_info["address"],

            "Order ID": invoice_info["order_id"],
            "Order Date": invoice_info["order_date"],

            "Invoice Date": invoice_info["invoice_date"],
            "Invoice Number": invoice_info["invoice_number"],

            "Product Title": product.get("title", "N/A"),
            "Quantity": product.get("quantity", 0),
            "Gross Amount": product.get("gross_amount", 0.0),
            "Discount": product.get("discount", 0.0),
            "Taxable Value": product.get("taxable_value", 0.0),
            "IGST": product.get("igst", 0.0),
            "Total": product.get("total", 0.0),
            "Total Items": totals["total_items"],
            
            "Subtotal": totals["sub_total"],
            "Total Discount": totals["total_discount"],
            "Total Taxable Value": totals["total_taxable_value"],
            "Total IGST": totals["total_igst"],
            "Grand Total": totals["grand_total"]
        }
        records.append(record)

    # return dataframe
    return pd.DataFrame(records)


In [28]:
json2df(output_data)

,Retailer Name,Retailer Address,Customer Name,Customer Address,Order ID,Order Date,Invoice Date,Invoice Number,Product Title,Quantity,Gross Amount,Discount,Taxable Value,IGST,Total,Total Items,Subtotal,Total Discount,Total Taxable Value,Total IGST,Grand Total
0,Tech-Connect Retail Private Limited,Marasandra and Madnahatti -Venkatapura Village...,Prafull Narayan Sonawane,"Talathi Colony near govind nagar plot no. 39, ...",OD329836817381305100,02-12-2023,02-12-2023,FAETLP2402126539,HRX by Hrithik Roshan Full Sleeve Solid Men Ja...,1,1079.0,0.0,963.39,115.61,1079.0,1,1129.0,-40.0,972.32,116.68,1089.0
1,Tech-Connect Retail Private Limited,Marasandra and Madnahatti -Venkatapura Village...,Prafull Narayan Sonawane,"Talathi Colony near govind nagar plot no. 39, ...",OD329836817381305100,02-12-2023,02-12-2023,FAETLP2402126539,Shipping And Handling Charges,1,50.0,-40.0,8.93,1.07,10.0,1,1129.0,-40.0,972.32,116.68,1089.0


In [97]:
# json2df(output).to_csv("prafull_invoice.csv")